In [1]:
import pandas as pd

from wrangle import wrangle_github_repos

from matplotlib.style import use
use('~/.matplotlib/gruvbox.mplstyle')

[nltk_data] Downloading package stopwords to /Users/stray/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/stray/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
df = wrangle_github_repos(new_pickles=True)

In [3]:
df.head(3)

,repository,original_readme,cleaned_readme,lemmatized_readme,original_char_length,cleaned_char_length,pct_char_removed,natural_language,programming_language
0,ieee8023/covid-chestxray-dataset,\n#### 🛑 Note: please do not claim diagnostic ...,note please claim diagnostic performance model...,note please claim diagnostic performance model...,10610,4170,39,en,Jupyter Notebook
1,geohot/corona,# Reverse engineering the coronavirus (SARS-Co...,reverse engineering coronavirus sars cov 2 sta...,reverse engineering coronavirus sars cov 2 sta...,13101,3883,29,en,Python
2,ahmadawais/corona-cli,"<h4 align=""center"">\n <a href=""https://gith...",track coronavirus disease covid 19 novel coron...,track coronavirus disease covid 19 novel coron...,15371,4855,31,en,JavaScript
